In [ ]:
# Задача для заряженной пластины

In [29]:
from dolfin import *
from mshr import *

nx, ny = 50, 50
plx, prx = 0.2, 0.8
u0, u1 = 0.0, 0.05

mesh = UnitSquareMesh(nx, ny)

p, q = Point(0.2, 1.), Point(0.8, 1.)
tol1 = 0.15
tol2 = 0.1
tol3 = 0.07

class Breaks(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], p.x(), tol1) and near(x[1], p.y(), tol2) or near(x[0], q.x(), tol1) and near(x[1], q.y(), tol2) or near(x[1], 1., tol3)

breaks = Breaks()

# Number of refinements
nor = 2

for i in range(nor):
    edge_markers = EdgeFunction("bool", mesh)
    breaks.mark(edge_markers, True)

    adapt(mesh, edge_markers)
    mesh = mesh.child()

#---------------refinement-end

def plate_boundary(x, on_boundary):
    return (plx < x[0] < prx) and near(x[1], 1.0)

def empty_boundary(x, on_boundary):
    return on_boundary and not plate_boundary(x, on_boundary)

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bcs = [DirichletBC(V, Constant(u1), plate_boundary),
       DirichletBC(V, Constant(u0), empty_boundary)]

a = inner(grad(u), grad(v)) * dx
L = Constant(0.0) * v * dx

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(u)
plot(u_e)

#plot(mesh)

interactive()

In [28]:
import numpy as np

Ve = V #FunctionSpace(mesh, 'Lagrange', 1)
u_e = Function(Ve)
summand = Expression('4/m*v*sin(m*(l+r)/2)*sin(m*(r-l)/2)*sin(m*x[0])*sinh(m*x[1])/sinh(m*b)', b=1.0, m=pi, l=plx, r=prx, v=u1)

for i in xrange(200):
    summand.m = pi * (i + 1)
    u_e.vector()[:] += interpolate(summand, Ve).vector()
    
mf = MeshFunction("size_t", mesh, mesh.topology().dim())
mf.set_all(1)

breaks.mark(mf, 0)

#v = interpolate(u, Ve)

print sqrt(assemble((u - u_e) ** 2 * dx)) #identical to errornorm

dX = dx(subdomain_data=mf)
error = assemble((u - u_e) ** 2 * dX(1))
print sqrt(error)

plot(abs(u - u_e), interactive=True)

0.000186274402706
4.87482384003e-05


<dolfin.cpp.io.VTKPlotter; proxy of <Swig Object of type 'std::shared_ptr< dolfin::VTKPlotter > *' at 0xa5cb8710> >

In [5]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri

triang = tri.Triangulation(*mesh.coordinates().reshape((-1, 2)).T,
                           triangles=mesh.cells())
Z = u.compute_vertex_values(mesh)

plt.figure()
plt.tricontour(triang, Z, )
plt.colorbar()
plt.show()

Вычисление невязки для разных шагов сетки и оценка скорости сходимости численного метода

In [32]:
from dolfin import *
from math import log as ln

plx, prx = 0.2, 0.8
u0, u1 = 0.0, 0.05
p, q = Point(0.2, 1.), Point(0.8, 1.)
tol1 = 0.1
tol2 = 0.1
tol3 = 0.05

class Breaks(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], p.x(), tol1) and near(x[1], p.y(), tol2) or near(x[0], q.x(), tol1) and near(x[1], q.y(), tol2) or near(x[1], 1., tol3)

def plate_boundary(x, on_boundary):
    return (plx < x[0] < prx) and near(x[1], 1.0)

def empty_boundary(x, on_boundary):
    return on_boundary and not plate_boundary(x, on_boundary)

breaks = Breaks()

def get_error(n):
    nx, ny = n, n
    mesh = UnitSquareMesh(nx, ny)

    nor = 0
    for i in range(nor):
        edge_markers = EdgeFunction("bool", mesh)
        breaks.mark(edge_markers, True)
        adapt(mesh, edge_markers)
        mesh = mesh.child()

    V = FunctionSpace(mesh, "Lagrange", 1)
    u = TrialFunction(V)
    v = TestFunction(V)

    bcs = [DirichletBC(V, Constant(u0), empty_boundary),
           DirichletBC(V, Constant(u1), plate_boundary)]

    a = inner(grad(u), grad(v)) * dx
    L = Constant(0.0) * v * dx

    u = Function(V)

    problem = LinearVariationalProblem(a, L, u, bcs)
    solver = LinearVariationalSolver(problem)
    solver.solve()

    W = FunctionSpace(mesh, 'Lagrange', 3)
    u_e = Function(W)

    summand = Expression('4/m*v*sin(m*(l+r)/2)*sin(m*(r-l)/2)*sin(m*x[0])*sinh(m*x[1])/sinh(m*b)', b=1.0, m=pi, l=plx, r=prx, v=u1)
    
    for i in xrange(200):
        summand.m = pi * (i + 1)
        u_e.vector()[:] += interpolate(summand, W).vector()
    
    mf = MeshFunction("size_t", mesh, mesh.topology().dim())
    mf.set_all(1)

    breaks.mark(mf, 0)

    u = interpolate(u, W)

    print n
    print sqrt(assemble((u - u_e) ** 2 * dx))
    dX = dx(subdomain_data=mf)
    error = assemble((u - u_e) ** 2 * dX(1))
    error_norm = sqrt(error)
    print error_norm
    
    return error_norm

In [33]:
N = [4, 8, 16, 32, 64]#, 128]#, 256]
h = [1. / n for n in N]
e = [get_error(n) for n in N]
r = [ln(e[i + 1] / e[i]) / ln(h[i + 1] / h[i])  for i in xrange(len(e) - 1)]
print r

4
0.00445580550155
0.00445580550155
8
0.00132337526268
0.00132337526268
16
0.00122776472871
0.000644668414516
32
0.000316514007203
9.0532360111e-05
64
0.000337147355982
0.000133435165435
[1.7514640423908698, 1.037593011612291, 2.8320518304924356, -0.5596334537590012]


In [7]:
print '\n'.join(['n={:>3} e={:.8f} r={:f}'.format(N[i], e[i], r[i]) for i in xrange(len(r))])

n=  4 e=0.00187144 r=4.577105
n=  8 e=0.00007840 r=-0.116347
n= 16 e=0.00008499 r=1.067510


In [ ]:
N = [8, 16, 32, 64]
h = [1. / n for n in N]
e = [get_error(n) for n in N]
r = [ln(e[i + 1] / e[i]) / ln(h[i + 1] / h[i])  for i in xrange(len(e) - 1)]
print r

# Эксперименты с сетками

In [ ]:
import numpy
import dolfin as df

def create_mesh(length, height, nx, ny, show=False):
    """
    Make a square mesh manually

    Should give exactly the same results as using the built in dolfin.RectangleMesh() class
    """
    # The number of mesh entities
    nvert = nx*ny
    ncell = 2*(nx-1)*(ny-1)

    # Positions of the vertices
    xpos = numpy.linspace(0, length, nx)
    ypos = numpy.linspace(0, height, ny)

    # Create the mesh and open for editing
    mesh = df.Mesh()
    editor = df.MeshEditor()
    editor.open(mesh, 2, 2)

    # Add the vertices (nodes)
    editor.init_vertices(nvert)
    i_vert = 0
    for x in xpos:
        for y in ypos:
            editor.add_vertex(i_vert, x, y)
            i_vert += 1

    # Add the cells (triangular elements)
    # Loop over the vertices and build two cells for each square
    # where the selected vertex is in the lower left corner
    editor.init_cells(ncell)
    i_cell = 0
    for ix in xrange(nx-1):
        for iy in xrange(ny-1):
            # Upper left triangle in this square
            i_vert0 = ix*ny + iy
            i_vert1 = ix*ny + iy+1
            i_vert2 = (ix+1)*ny + iy + 1
            editor.add_cell(i_cell, i_vert0, i_vert1, i_vert2)
            i_cell += 1

            # Lower right triangle in this square
            i_vert0 = ix*ny + iy
            i_vert1 = (ix+1)*ny + iy+1
            i_vert2 = (ix+1)*ny + iy
            editor.add_cell(i_cell, i_vert0, i_vert1, i_vert2)
            i_cell += 1

    # Close the mesh for editing
    editor.close()
    print 'Created mesh with %d vertices and %d cells' % (nvert, ncell)

    if show:
        df.plot(mesh)
        df.interactive()

    return mesh

mesh = create_mesh(5, 3, 10, 10)

df.plot(mesh)
df.interactive()

In [1]:
from dolfin import *

mesh = UnitSquareMesh(20, 20)

# Break point
p, q = Point(0.2, 1.), Point(0.8, 1.)
tol = 0.05

# Selecting edges to refine
class Border(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[0], p.x(), tol) and near(x[1], p.y(), tol) or
    near(x[0], q.x(), tol) and near(x[1], q.y(), tol)) and on_boundary

Border = Border()

# Number of refinements
nor = 5

for i in range(nor):
    edge_markers = EdgeFunction("bool", mesh)
    Border.mark(edge_markers, True)

    adapt(mesh, edge_markers)
    mesh = mesh.child()

plot(mesh, interactive=True)


<dolfin.cpp.io.VTKPlotter; proxy of <Swig Object of type 'std::shared_ptr< dolfin::VTKPlotter > *' at 0xa70bf638> >

In [11]:
from dolfin import *

mesh = UnitCubeMesh(5, 2, 3)

plot(mesh, interactive=True)

<dolfin.cpp.io.VTKPlotter; proxy of <Swig Object of type 'std::shared_ptr< dolfin::VTKPlotter > *' at 0x99886788> >

In [13]:
from dolfin import *
from mshr import *

mesh = generate_mesh(Rectangle(Point(0, 0), Point(1, 1)), 5)

plot(mesh)

interactive()

In [13]:
import numpy
from mshr import *

Theta = 2 * pi
a, b = 3., 5.0
nr = 10  # divisions in r direction
nt = 20  # divisions in theta direction
mesh = UnitSquareMesh(10, 10, 'crossed')
#mesh = generate_mesh(Rectangle(Point(0, 0), Point(1, 1)), 10)
#mesh = RectangleMesh(nr, nt, digonal='crossed')

# First make a denser mesh towards r=a
x = mesh.coordinates()[:,0]
y = mesh.coordinates()[:,1]
s = 1

def denser(x, y):
    return [a + (b-a)*((x-a)/(b-a))**s, y]

x_bar, y_bar = denser(x, y)
xy_bar_coor = numpy.array([x_bar, y_bar]).transpose()
mesh.coordinates()[:] = xy_bar_coor
plot(mesh, title='stretched mesh')

def cylinder(r, s):
    return [r*numpy.cos(Theta*s), r*numpy.sin(Theta*s)]

x_hat, y_hat = cylinder(x, y)
xy_hat_coor = numpy.array([x_hat, y_hat]).transpose()
mesh.coordinates()[:] = xy_hat_coor
plot(mesh, title='hollow cylinder')
interactive()

# Задача с непрерывными граничными условиями

In [34]:
from dolfin import *

nx, ny = 15, 15

mesh = UnitSquareMesh(nx, ny)

def left_boundary(x, on_boundary):
    return near(x[0], 0.)

def right_boundary(x, on_boundary):
    return near(x[0], 1.)

def bottom_boundary(x, on_boundary):
    return near(x[1], 0.)

def top_boundary(x, on_boundary):
    return near(x[1], 1.)


V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

bcs = [DirichletBC(V, Expression('1-x[1]*x[1]'), left_boundary),
       DirichletBC(V, Expression('2-x[1]*x[1]'), right_boundary),
       DirichletBC(V, Expression('1+x[0]*x[0]'), bottom_boundary),
       DirichletBC(V, Expression('x[0]*x[0]'), top_boundary)]

a = inner(grad(u), grad(v)) * dx
L = Constant(0.0) * v * dx

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(u)

E = -project(grad(u), VectorFunctionSpace(mesh, "Lagrange", 1))

plot(E)

u_e = Expression('1+x[0]*x[0]-x[1]*x[1]')

plot(u - u_e)

Ve = FunctionSpace(mesh, 'Lagrange', 5)
print 'Error norm:', errornorm(u_e, u, norm_type='l2')

interactive()

Calling FFC just-in-time (JIT) compiler, this may take some time.
Error norm: 0.000468485579284


In [1]:
from dolfin import *
from mshr import *

r, R = 0.1, 3.
count = 30

mesh = generate_mesh(Circle(Point(0., 0.), R, 30) - Circle(Point(0., 0.), r, 20), count)

plot(mesh)

#'''
def outer_boundary(x, on_boundary):
    return on_boundary and (x[0]*x[0] + x[1]*x[1] > r + 0.1)

def inner_boundary(x, on_boundary):
    return on_boundary and (x[0]*x[0] + x[1]*x[1] < r + 0.1)
#'''

V = FunctionSpace(mesh, "Lagrange", 1)
u = TrialFunction(V)
v = TestFunction(V)

#'''
bcs = [DirichletBC(V, Constant(1.), outer_boundary),
       DirichletBC(V, Constant(10.), inner_boundary)]
#'''

#bcs = DirichletBC(V, Expression('1/sqrt(x[0]*x[0]+x[1]*x[1])'), lambda x, on_boundary: on_boundary)

a = inner(grad(u), grad(v)) * dx
L = Constant(0.0) * v * dx

u = Function(V)

problem = LinearVariationalProblem(a, L, u, bcs)
solver = LinearVariationalSolver(problem)
solver.solve()

plot(u)

E = -project(grad(u), VectorFunctionSpace(mesh, "Lagrange", 1))

plot(E)

interactive()

In [2]:
import scitools.easyviz as ev
import scitools.BoxField

u_box = scitools.BoxField.dolfin_function2BoxField(u, mesh, (20, 88))
ev.contour(u_box.grid.coorv[0], u_box.grid.coorv[1], u_box.values, 5)
ev.savefig('hey.png')

In [14]:
from dolfin import *
from math import log as ln

def plate_boundary(x, on_boundary):
    return (plx < x[0] < prx) and near(x[1], 1.0)

def empty_boundary(x, on_boundary):
    return on_boundary and not plate_boundary(x, on_boundary)

def get_error(n):
    nx, ny = n, n
    
    mesh = UnitSquareMesh(nx, ny)
    
    V = FunctionSpace(mesh, "Lagrange", 1)
    u = TrialFunction(V)
    v = TestFunction(V)

    bcs = [DirichletBC(V, Expression('1-x[1]*x[1]'), left_boundary),
           DirichletBC(V, Expression('2-x[1]*x[1]'), right_boundary),
           DirichletBC(V, Expression('1+x[0]*x[0]'), bottom_boundary),
           DirichletBC(V, Expression('x[0]*x[0]'), top_boundary)]

    a = inner(grad(u), grad(v)) * dx
    L = Constant(0.0) * v * dx

    u = Function(V)

    problem = LinearVariationalProblem(a, L, u, bcs)
    solver = LinearVariationalSolver(problem)
    solver.solve()

    u_e = Expression('1+x[0]*x[0]-x[1]*x[1]')
    
    return errornorm(u_e, u, norm_type='l2')

In [15]:
N = [4, 8, 16, 32, 64, 128, 256]
h = [1. / n for n in N]
e = [get_error(n) for n in N]
r = [ln(e[i + 1] / e[i]) / ln(h[i + 1] / h[i])  for i in xrange(len(e) - 1)]
print r

[1.999999999999993, 1.9999999999999825, 1.9999999999999727, 1.9999999999999434, 1.9999999999998712, 1.9999999999997309]


In [22]:
print '\n'.join(['n={:>3} e={:.8f} r={:.15f}'.format(N[i], e[i], r[i]) for i in xrange(len(r))])
print 'n={:>3} e={:.8f}'.format(N[-1], e[-1])

n=  4 e=0.00658808 r=1.999999999999993
n=  8 e=0.00164702 r=1.999999999999982
n= 16 e=0.00041175 r=1.999999999999973
n= 32 e=0.00010294 r=1.999999999999943
n= 64 e=0.00002573 r=1.999999999999871
n=128 e=0.00000643 r=1.999999999999731
n=256 e=0.00000161
